# Описание проекта

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Доступны пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Проанализируем возможную прибыль и риски техникой *Bootstrap.* Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from scipy import stats as st

import warnings 
warnings.filterwarnings('ignore')

In [70]:
df_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

df_0.info()
df_0.head()
df_0.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [71]:
df_1.info()
df_1.head()
df_1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [72]:
df_2.info()
df_2.head()
df_2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [73]:
#поиск дубликатов
df_0.duplicated().sum()
df_1.duplicated().sum()
df_2.duplicated().sum()

0

Данные проверили: аномалий, пропусков и дубликатов нет.

In [74]:
df_0.corr()
a = sns.pairplot(df_0)

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [75]:
df_1.corr()
a = sns.pairplot(df_1)

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [76]:
df_2.corr()
a = sns.pairplot(df_2)

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Признаки не сильно коррелируются между собой, поэтому все оставляем.

# 2. Обучение и проверка модели

Обучим и проверим модель для каждого региона: разобьем данные на обучающую и валидационную выборки в соотношении 75:25, обучим модель и сделаем предсказания на валидационной выборке.

## Регион 0

In [77]:
x = df_0.drop(['product', 'id'], axis=1)
y = df_0['product']

In [78]:
x_train0, x_valid0, y_train0, y_valid0 = train_test_split(x, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(x_train0)
x_train0 = scaler.transform(x_train0)
x_valid0 = scaler.transform(x_valid0)

print(x_train0.shape) 
print(x_valid0.shape) 
print(y_train0.shape) 
print(y_valid0.shape) 

(75000, 3)
(25000, 3)
(75000,)
(25000,)


Признаки масштабировали и проконтролировали размеры новых выборок. Построим модель и повторим шаги для других регионов.

In [79]:
model_0 = LinearRegression()
model_0.fit(x_train0, y_train0)
predictions_0 = model_0.predict(x_valid0)

mse_0 = mean_squared_error(y_valid0, predictions_0)
print('Средний запас предсказанного сырья', predictions_0.mean())
print('RMSE:', mse_0 ** 0.5)

Средний запас предсказанного сырья 92.39879990657768
RMSE: 37.75660035026169


## Регион 1

In [101]:
x1 = df_1.drop(['product', 'id'], axis=1)
y1 = df_1['product']

x_train1, x_valid1, y_train1, y_valid1 = train_test_split(x1, y1, test_size=0.25, random_state=42)

print(x_train1.shape) 
print(x_valid1.shape) 
print(y_train1.shape) 
print(y_valid1.shape) 

model_1 = LinearRegression()
model_1.fit(x_train1, y_train1)
predictions_1 = model_1.predict(x_valid1)

mse_1 = mean_squared_error(y_valid1, predictions_1)
print('Средний запас предсказанного сырья', predictions_1.mean())
print('RMSE:', mse_1 ** 0.5)

(75000, 3)
(25000, 3)
(75000,)
(25000,)
Средний запас предсказанного сырья 68.71287803913764
RMSE: 0.8902801001028828


## Регион 2

In [105]:
x2 = df_2.drop(['product', 'id'], axis=1)
y2 = df_2['product']

x_train2, x_valid2, y_train2, y_valid2 = train_test_split(x2, y2, test_size=0.25, random_state=42)

print(x_train2.shape)
print(x_valid2.shape)
print(y_train2.shape)
print(y_valid2.shape)

model = LinearRegression()
model.fit(x_train2, y_train2)
predictions_2 = model.predict(x_valid2)

mse_2 = mean_squared_error(y_valid2, predictions_2)
print('Средний запас предсказанного сырья', predictions_2.mean())
print('RMSE:', mse_2 ** 0.5)

(75000, 3)
(25000, 3)
(75000,)
(25000,)
Средний запас предсказанного сырья 94.77102387765939
RMSE: 40.145872311342174


Данные разбили на выборки, обучили модель линейной регрессии и сделали предсказания на валидационных наборах данных.
Адекватное значение RMSE получили только в случае региона 1 — 0.89

# 3. Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных, а именно:
- При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [82]:
points = 500
best_points = 200
budget = 10000000000
profit_barrel = 450
profit_unit = 450000
probability_of_loss = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [83]:
x = budget / (best_points * profit_unit)
x

111.11111111111111

In [84]:
print( 'Средний запас региона 0: {:.2f}'.format(df_0['product'].mean()))
print('Средний запас региона 1: {:.2f}'.format(df_1['product'].mean()))
print('Средний запас региона 2: {:.2f}'.format(df_2['product'].mean()))

Средний запас региона 0: 92.50
Средний запас региона 1: 68.83
Средний запас региона 2: 95.00


Достаточный объем сырья для разработки скважины — 111 тысяч баррелей. Есть точка безубыточности) <br>
Сравнительно с объемами по всем регионам в порядке убывания:
- Регион 2 — 95.00
- Регион 0 — 92.50
- Регион 1 — 68.83

# 4. Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберем скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины.
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [95]:
def revenue(target, pred, count):
    pred = pd.Series(pred, index=target.index)
    pred_sorted = pred.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return selected.sum() * profit_unit - budget

Посчитаем риски и прибыль для каждого региона:
- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [109]:
state = np.random.RandomState(42)

values = []
for i in range(1000):
    target_subsample = y_valid0.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    pred_subsample = predictions_0[target_subsample.index]
    values.append(revenue(target_subsample, pred_subsample, 200))
    
values = pd.Series(values)
mean = values.mean()
negative_profit_chance = (values < 0).mean()
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Средняя прибыль региона {:.2f} млн.руб'.format(mean/1000000))
print('95%-й доверительный интервал ({:.2f},{:.2f})'.format(lower/1000000, upper/1000000))
print('Риск убытков {:.2f}%'.format(negative_profit_chance * 100))

Средняя прибыль региона 427.85 млн.руб
95%-й доверительный интервал (-97.25,954.22)
Риск убытков 5.50%


In [110]:
state = np.random.RandomState(42)

values = []
for i in range(1000):
    target_subsample = y_valid1.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    pred_subsample = predictions_1[target_subsample.index]
    values.append(revenue(target_subsample, pred_subsample, 200))
    
values = pd.Series(values)
mean = values.mean()
negative_profit_chance = (values < 0).mean()
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Средняя прибыль региона {:.2f} млн.руб'.format(mean/1000000))
print('95%-й доверительный интервал ({:.2f},{:.2f})'.format(lower/1000000, upper/1000000))
print('Риск убытков {:.2f}%'.format(negative_profit_chance * 100))

Средняя прибыль региона 511.36 млн.руб
95%-й доверительный интервал (98.87,940.72)
Риск убытков 0.90%


In [111]:
state = np.random.RandomState(42)

values = []
for i in range(1000):
    target_subsample = y_valid2.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    pred_subsample = predictions_2[target_subsample.index]
    values.append(revenue(target_subsample, pred_subsample, 200))
    
values = pd.Series(values)
mean = values.mean()
negative_profit_chance = (values < 0).mean()
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Средняя прибыль региона {:.2f} млн.руб'.format(mean/1000000))
print('95%-й доверительный интервал ({:.2f},{:.2f})'.format(lower/1000000, upper/1000000))
print('Риск убытков {:.2f}%'.format(negative_profit_chance * 100))

Средняя прибыль региона 402.58 млн.руб
95%-й доверительный интервал (-137.16,929.89)
Риск убытков 7.40%


Проанализированы пробы нефти из 3 регионов. Спрогнозировали возможную прибыль и риски техникой Bootstrap:

Регион 1
- Средняя прибыль региона 427.85 млн.руб
- 95%-й доверительный интервал (-97.25,954.22)
- Риск убытков 5.50%

Регион 2
- Средняя прибыль региона 511.36 млн.руб
- 95%-й доверительный интервал (98.87,940.72)
- Риск убытков 0.90%

Регион 3
- Средняя прибыль региона 402575607.51
- 95%-й доверительный интервал (-137.16,929.89)
- Риск убытков 7.40%

По результатам, рекомендуем бурить новые скважины в регионе 1, так как риск убытков в данном случае минимальный, суммарная прибыль отобранных скважин максимальна и положительный доверительный интервал.